# 4-Oscar Prediction with AutoML
After out dataframe has been assemlbed (see scraping and table_assembling) notebooks we have the data we need to make predictions on the Best Picture winner. [AutoML](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html) represents a quick, but powerful route though the Machine Learning process. H2O's AutoML runs many models through the dataset and using cross-validation, picks the best one. For my purposes I use it to confirm/compare to the Preferential Balloting Random Forest model I created.
If you are gunning to win your office's Oscar pool, scroll down to see the results.

In [1]:
import pandas as pd
import numpy as np
import h2o

In [10]:
from h2o.estimators import H2OXGBoostEstimator

# Machine Learning - Using h2o Auto ML

In [2]:
full_table = pd.read_csv('../data/processed_results/extended_df.csv')
full_table=full_table.drop('Unnamed: 0', axis=1)
full_table

,year,film,wiki,winner,rating,numVotes,worldwide_box_office,action,adventure,animation,...,nom_pga,winner_pga,nom_bafta,winner_bafta,nom_dga,winner_dga,nom_sag,winner_sag,nom_cannes,winner_cannes
0,1927,Wings,/wiki/Wings_(1927_film),True,7.3,13576.0,$746,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1927,7th Heaven,/wiki/7th_Heaven_(1927_film),False,5.2,26223.0,"$79,808",0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1927,The Racket,/wiki/The_Racket_(1928_film),False,6.7,3149.0,"$21,733,230",0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1928,The Broadway Melody,/wiki/The_Broadway_Melody,True,5.6,7605.0,"$223,723",0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1928,Alibi,/wiki/Alibi_(1929_film),False,7.4,391.0,"$42,915",0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,2022,The Fabelmans,/wiki/The_Fabelmans,False,7.6,85709.0,"$45,164,110",0,0,0,...,1,0,0,0,1,0,1,0,0,0
547,2022,Tár,/wiki/T%C3%A1r,False,7.5,69684.0,"$27,541,681",0,0,0,...,1,0,1,0,1,0,0,0,0,0
548,2022,Top Gun: Maverick,/wiki/Top_Gun:_Maverick,False,8.3,577408.0,"$1,493,491,858",0,0,0,...,1,0,0,0,1,0,0,0,0,0
549,2022,Triangle of Sadness,/wiki/Triangle_of_Sadness,False,7.3,128812.0,"$25,615,870",0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.241-b07, mixed mode)
  Starting server from C:\Users\Aleksandra Czaplak\AppData\Local\Programs\Python\Python39\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\ALEKSA~1\AppData\Local\Temp\tmp0g3__ksz
  JVM stdout: C:\Users\ALEKSA~1\AppData\Local\Temp\tmp0g3__ksz\h2o_Aleksandra_Czaplak_started_from_python.out
  JVM stderr: C:\Users\ALEKSA~1\AppData\Local\Temp\tmp0g3__ksz\h2o_Aleksandra_Czaplak_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,07 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,22 days
H2O_cluster_name:,H2O_from_python_Aleksandra_Czaplak_8vsa57
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.761 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


First Year of Existance. This data will be used below
- golden_globes 1943
- pga 1989
- bafta 1960
- dga 1948
- sag 1995
- cannes 1970

In [4]:
# I pick a min_year where the awards shows will be relevant
min_year = 1995

# H2O's Auto ML

In [5]:
# Auto ML uses Cross Validation, so we do not specifiy a validation set
train = full_table.loc[((full_table['year'] < 2022) & (full_table['year'] >= min_year))]

print('training set contains:', train.shape[0], 'movies')

training set contains: 184 movies


In [6]:
#train = train.drop(['index', '[]'], axis=1)
train.columns

Index(['year', 'film', 'wiki', 'winner', 'rating', 'numVotes',
       'worldwide_box_office', 'action', 'adventure', 'animation', 'biography',
       'comedy', 'crime', 'documentary', 'drama', 'family', 'fantasy',
       'film-noir', 'history', 'horror', 'music', 'musical', 'mystery',
       'romance', 'sci-fi', 'sport', 'thriller', 'war', 'western',
       'nominations', 'Oscar_win', 'nom_gg_drama', 'winner_gg_drama',
       'nom_gg_comedy', 'winner_gg_comedy', 'nom_pga', 'winner_pga',
       'nom_bafta', 'winner_bafta', 'nom_dga', 'winner_dga', 'nom_sag',
       'winner_sag', 'nom_cannes', 'winner_cannes'],
      dtype='object')

In [6]:
print(type(train))

<class 'pandas.core.frame.DataFrame'>


In [7]:
from h2o.automl import H2OAutoML, get_leaderboard

# Import a sample binary outcome train/test set into H2O
train1 = h2o.H2OFrame(train)

# Identify predictors and response
predictors = ['year', 'rating', 'numVotes',
       'worldwide_box_office', 'action', 'adventure',
       'animation', 'biography', 'comedy', 'crime', 'documentary', 'drama',
       'family', 'fantasy', 'film-noir', 'history', 'horror', 'music',
       'musical', 'mystery', 'romance', 'sci-fi', 'sport', 'thriller', 'war',
       'western', 'nominations', 'nom_gg_drama',
       'winner_gg_drama', 'nom_gg_comedy', 'winner_gg_comedy', 'nom_pga',
       'winner_pga', 'nom_bafta', 'winner_bafta', 'nom_dga', 'winner_dga',
       'nom_sag', 'winner_sag', 'nom_cannes', 'winner_cannes']

x = predictors
y = 'Oscar_win'

# For binary classification, response should be a factor
train1[y] = train1[y].asfactor()

# Run AutoML for 100 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=100, seed=1
                , keep_cross_validation_predictions= True
               , exclude_algos = ['StackedEnsemble'])

aml.train(x=x, y=y, training_frame=train1)

# AutoML Leaderboard
lb = aml.leaderboard

# Optionally edd extra model information to the leaderboard
lb = get_leaderboard(aml, extra_columns='ALL')

# Print all rows (instead of default 10 rows)
lb.head(rows=lb.nrows)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
14:45:14.508: AutoML: XGBoost is not available; skipping it.
14:45:14.719: _train param, Dropping bad and constant columns: [nom_cannes, winner_cannes, film-noir, worldwide_box_office, documentary]


14:45:16.395: _train param, Dropping bad and constant columns: [nom_cannes, winner_cannes, film-noir, worldwide_box_office, documentary]
14:45:16.396: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 184.0.
14:45:16.405: _train param, Dropping bad and constant columns: [nom_cannes, winner_cannes, film-noir, worldwide_box_office, documentary]

██
14:45:18.631: _train param, Dropping bad and constant columns: [nom_cannes, winner_cannes, film-noir, worldwide_box_office, documentary]
14:45:19.866: _train param, Dropping bad and constant columns: [nom_cannes, winner_cannes, film-noir, worldwide_box_of

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms,algo
DeepLearning_grid_1_AutoML_1_20230520_144514_model_15,0.791106,1.09683,0.433063,0.259023,0.418761,0.175361,2607,1.49015,DeepLearning
DeepLearning_grid_3_AutoML_1_20230520_144514_model_1,0.772588,0.597379,0.423092,0.23732,0.340284,0.115794,10574,0.40056,DeepLearning
DeepLearning_grid_1_AutoML_1_20230520_144514_model_3,0.747818,1.73519,0.285581,0.29087,0.44639,0.199264,2600,0.247966,DeepLearning
DeepLearning_grid_3_AutoML_1_20230520_144514_model_4,0.740269,0.563238,0.56181,0.296886,0.34356,0.118033,5737,0.207271,DeepLearning
DeepLearning_grid_3_AutoML_1_20230520_144514_model_14,0.717622,0.544596,0.288007,0.299835,0.376135,0.141477,10366,0.264486,DeepLearning
DeepLearning_grid_1_AutoML_1_20230520_144514_model_22,0.713966,1.13228,0.364345,0.322364,0.39171,0.153437,3805,0.256387,DeepLearning
DeepLearning_1_AutoML_1_20230520_144514,0.704647,0.550712,0.321968,0.349847,0.399602,0.159682,175,0.267711,DeepLearning
DeepLearning_grid_1_AutoML_1_20230520_144514_model_8,0.704411,0.751851,0.278718,0.289691,0.393847,0.155116,9067,0.258215,DeepLearning
DeepLearning_grid_3_AutoML_1_20230520_144514_model_21,0.686954,0.634938,0.239659,0.30243,0.399271,0.159417,2917,0.105547,DeepLearning
DeepLearning_grid_2_AutoML_1_20230520_144514_model_20,0.68318,0.604861,0.26804,0.343241,0.370924,0.137585,2724,0.287551,DeepLearning


In [ ]:
# Import a sample binary outcome train/test set into H2O
train1 = h2o.H2OFrame(train)

# Identify predictors and response
predictors = ['year', 'rating', 'numVotes',
       'worldwide_box_office', 'action', 'adventure',
       'animation', 'biography', 'comedy', 'crime', 'documentary', 'drama',
       'family', 'fantasy', 'film-noir', 'history', 'horror', 'music',
       'musical', 'mystery', 'romance', 'sci-fi', 'sport', 'thriller', 'war',
       'western', 'nominations', 'nom_gg_drama',
       'winner_gg_drama', 'nom_gg_comedy', 'winner_gg_comedy', 'nom_pga',
       'winner_pga', 'nom_bafta', 'winner_bafta', 'nom_dga', 'winner_dga',
       'nom_sag', 'winner_sag', 'nom_cannes', 'winner_cannes']

x = predictors
y = 'Oscar_win'

# For binary classification, response should be a factor
train1[y] = train1[y].asfactor()


# Split the dataset into train and test sets
train1, test = train1.split_frame(ratios=[0.8])

# Define the XGBoost estimator
xgboost = H2OXGBoostEstimator()

# Train the XGBoost model
xgboost.train(x=x, y=y, training_frame=train1)

# Evaluate the model on the test set
performance = xgboost.model_performance(test_data=test)
print(performance)

In [8]:
top_model = aml.leader
top_model

Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_1_AutoML_1_20230520_144514_model_15


Status of Neuron Layers: predicting Oscar_win, 2-class classification, bernoulli distribution, CrossEntropy loss, 782 weights/biases, 16,4 KB, 257 600 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate              rate_rms              momentum    mean_weight           weight_rms           mean_bias             bias_rms
--  -------  -------  ----------------  ---------  ----  ----  ---------------------  --------------------  ----------  --------------------  -------------------  --------------------  -------------------
    1        36       Input             5.0
    2        20       RectifierDropout  10.0       0.0   0.0   0.0022078270879218407  0.003123491071164608  0.0         0.09029229445762516   0.26058220863342285  0.2456089317952399    0.21000957489013672
    3        2        Softmax                      0.0   0.0   0.0018783423904096708  0.003721131943166256  0.0         -0.05552845075726509  1.3402414321899414   -0.03331026685106242  0.5842199325561523

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.0015749634753609823
RMSE: 0.03968580949610304
LogLoss: 0.010643031295491859
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.7357392318050676
       0    1    Error    Rate
-----  ---  ---  -------  -----------
0      157  0    0        (0.0/157.0)
1      0    27   0        (0.0/27.0)
Total  157  27   0        (0.0/184.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.735739     1         25
max f2                       0.735739     1         25
max f0point5                 0.735739     1         25
max accuracy                 0.735739     1         25
max precision                1            1         0
max recall                   0.735739     1         25
max specificity              1            1         0
max absolute_mcc             0.735739     1         25
max min_per_class_accuracy   0.735739     1         25
max mean_per_class_accuracy  0.735739     1         25
max tns                      1            157       0
max fns                      1            25        0
max fps                      3.34352e-20  157       182
max tps                      0.735739     27        25
max tnr                      1            1         0
max fnr                      1            0.925926  0
max fpr                      3.34352e-20  1         182
max tpr                      0.735739     1         25

Gains/Lift Table: Avg response rate: 14,67 %, avg score: 14,62 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0108696                   0.999999           6.81481  6.81481            1                1            1                           1                   0.0740741       0.0740741                  581.481  581.481            0.0740741
2        0.0217391                   0.999988           6.81481  6.81481            1                0.999996     1                           0.999998            0.0740741       0.148148                   581.481  581.481            0.148148
3        0.0326087                   0.999891           6.81481  6.81481            1                0.999974     1                           

In [9]:
model = h2o.load_model('./additional_data/model/DeepLearning_grid_2_AutoML_6_20230519_91906_model_4')

H2OResponseError: Server error water.exceptions.H2OIllegalArgumentException:
  Error: Illegal argument: dir of function: importModel: water.api.FSIOException: FS IO Failure: 
 accessed path : file:/c:/Users/Aleksandra%20Czaplak/Desktop/oscars_ml/oscar_predictions/additional_data/./additional_data/model/DeepLearning_grid_2_AutoML_6_20230519_91906_model_4 msg: File not found
  Request: POST /99/Models.bin/
    data: {'dir': './additional_data/model/DeepLearning_grid_2_AutoML_6_20230519_91906_model_4'}


## Predict the winner

In [13]:
# Predict on 2019's films
test = full_table.loc[(full_table['year'] == 2021)]

# Import a binary outcome train/test set into H2O
test = h2o.H2OFrame(test)

# For binary classification, response should be a factor
test[y] = test[y].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [14]:
preds = top_model.predict(test)

preds

deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


predict,p0,p1
0,0.999976,2.37118e-05
1,0.00165605,0.998344
0,0.995174,0.00482619
0,1,1.74511e-07
0,0.995249,0.00475117
0,0.999972,2.78123e-05
0,0.999979,2.0885e-05
0,1,9.07248e-09
0,0.999955,4.5392e-05
0,0.999999,1.10776e-06


In [15]:
test['pred'] = preds['predict']
test['probA'] = preds['p1']
test_pd = test.as_data_frame(use_pandas=True)

In [16]:
final_rankings = test_pd[['film','probA']].sort_values('probA', ascending = False)
final_rankings['%_confidence'] = final_rankings['probA']/final_rankings['probA'].sum() * 100
final_rankings

,film,probA,%_confidence
1,CODA,9.983440e-01,9.903809e+01
2,Belfast,4.826187e-03,4.787692e-01
4,Drive My Car,4.751175e-03,4.713278e-01
8,Nightmare Alley,4.539204e-05,4.502998e-03
5,Dune,2.781235e-05,2.759051e-03
0,West Side Story,2.371181e-05,2.352268e-03
6,King Richard,2.088495e-05,2.071837e-03
9,The Power of the Dog,1.107759e-06,1.098923e-04
3,Don't Look Up,1.745112e-07,1.731192e-05
7,Licorice Pizza,9.072485e-09,9.000120e-07


# And the Oscar goes to...

In [ ]:
bp_winner = np.array(final_rankings.reset_index())[0][1].split('(')[0].strip()
print(f'And the Oscar goes to...\n🎉🏆{bp_winner}🏆🎉')